In [ ]:
#라이브러리 불러오기
import os
from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import shutil
import math
import time

import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
#본 함수

def dataset_labeling(train_img,test_img,train_emo,test_emo,emo_data,each_emo_file_number,emo_file_starter,filename_para):
  #global emo_happy_starter,emo_surprise_starter,emo_neutral_starter,emo_fear_starter,emo_disgust_starter,emo_anger_starter,emo_sad_starter
  #global train_img,test_img,number,emo_happy_starter,emo_surprise_starter,emo_neutral_numberstarter,emo_fear_starter,emo_disgust_starter,emo_anger_starter,emo_sad_starter
  train_ratio = 1178
  if emo_file_starter <= train_ratio: #train 데이터셋 생성
    if train_img is None:
      train_img = import_frame_from_video(filename_para)
      train_emo.append(emo_data) #감정 데이터 생성
      print('dataset_labeling*',emo_file_starter)
    else:
      train_img, train_emo = import_frame_from_video_and_append(train_img, train_emo,emo_data, emo_file_starter,filename_para) #프레임을 하나씩 불러오고 list를 합침. 영상 하나가 끝나면 emo_file_starter가 +1이됨.
      emo_file_starter = emo_file_starter + 1
  elif emo_file_starter < each_emo_file_number: #test 데이터셋 생성
    if test_img is None:
      test_img = import_frame_from_video(filename_para)
      test_emo.append(emo_data)
      print('dataset_labeling**',emo_file_starter)
    else:
      test_img, test_emo = import_frame_from_video_and_append(test_img, test_emo,emo_data, emo_file_starter,filename_para)
      emo_file_starter = emo_file_starter + 1
  elif emo_file_starter == each_emo_file_number:
    test_img = import_frame_from_video_and_append(test_img, test_emo, emo_data,emo_file_starter,filename_para)
    emo_file_starter = 1
    print('dataset_labeling***',emo_file_starter)
  print('emo_file_starter = ',emo_file_starter)
  print('emo_happy_starter = ',emo_happy_starter)
  print('emo_surprise_starter',emo_surprise_starter)
  print('emo_neutral_starter',emo_neutral_starter)
  print('emo_fear_starter',emo_fear_starter)
  print('emo_disgust_starter',emo_disgust_starter)
  print('emo_anger_starter',emo_anger_starter)
  print('emo_sad_starter',emo_sad_starter)
  print('train_img.shape = ',train_img.shape)
  print('len(train_emo) = ',len(train_emo))
  if not test_img is None:
    print('test_img.shape = ',test_img.shape)
    print('len(test_emo) = ',len(test_emo))
  print('\n')
  print('filename',filename)
  print('emo_data',emo_data)
  print("time :", time.time() - start)
  np.save('/content/drive/MyDrive/save_numpy/train_img', train_img) # train_img.npy
  np.save('/content/drive/MyDrive/save_numpy/test_img', test_img) # test_img.npy
  np.save('/content/drive/MyDrive/save_numpy/train_emo', train_emo) # train_emo.npy
  np.save('/content/drive/MyDrive/save_numpy/test_emo', test_emo) # test_emo.npy
  return train_img, test_img, train_emo, test_emo, emo_file_starter
  

def import_frame_from_video_and_append(list_img, list_emo,emo_data, emo_file_starter,filename_para):
  #global emo_file_starter,emo_happy_starter,emo_surprise_starter,emo_neutral_numberstarter,emo_fear_starter,emo_disgust_starter,emo_anger_starter,emo_sad_starter
  count = 0 # fps 조절
  cap = cv2.VideoCapture("/content/drive/MyDrive/data/"+filename_para) #파일 불러오기
  while cap.isOpened(): #프레임단위로 불러오기
    ret, frame = cap.read()
    if not ret: # ret: read가 제대로 되었는지 확인하는 변수로, 이게 False라고 뜨면 영상이 끝난거라 while 문 멈추기
      break
    cropped_img = frame[:, 420:1500] #프레임의 양 옆을 잘라서 1:1로 만듦
    frame_resize = cv2.resize(cropped_img, dsize=(180, 180)) #프레임크기를 180*180으로 만듬
    frame_resize = frame_resize/255 #값을 0~1로 만듦.
    frame_resize = frame_resize[np.newaxis,:]
    list_img =  np.concatenate((list_img, frame_resize), axis=0) #리스트 합치기
    list_emo.append(emo_data) #감정 데이터 생성
    count += 60 # i.e. at 30 fps, this advances one second
    cap.set(1, count)
    print('*****',count)
  emo_file_starter = emo_file_starter + 1
  return list_img, list_emo

def import_frame_from_video(filename_para):
  cap = cv2.VideoCapture("/content/drive/MyDrive/data/"+filename_para) #파일 불러오기
  ret, frame = cap.read()
  cropped_img = frame[:, 420:1500] #프레임의 양 옆을 잘라서 1:1로 만듦
  frame_resize = cv2.resize(cropped_img, dsize=(180, 180)) #프레임크기를 180*180으로 만듬
  frame_resize = frame_resize/255 #값을 0~1로 만듦.
  frame_resize = frame_resize[np.newaxis,:]
  return frame_resize


def how_many_each_emo_number():
  global emo_happy_number,emo_surprise_number,emo_neutral_number,emo_fear_number,emo_disgust_number, emo_anger_number,emo_sad_number
  emo_happy_number = 1
  emo_surprise_number = 1
  emo_neutral_number = 1
  emo_fear_number = 1
  emo_disgust_number = 1
  emo_anger_number = 1
  emo_sad_number = 1
  for filename in file_list:
    filenum = filename[4:7]
    filenum = filenum.lstrip("0")
    filenum = int(filenum) #형식 변환
    if filenum <= 50:
      emo_happy_number = emo_happy_number + 1
    elif filenum <= 100:
      emo_surprise_number = emo_surprise_number + 1
    elif filenum <= 150:
      emo_neutral_number = emo_neutral_number + 1
    elif filenum <= 200:
      emo_fear_number = emo_fear_number + 1
    elif filenum <= 250:
      emo_disgust_number = emo_disgust_number + 1
    elif filenum <= 300:
      emo_anger_number = emo_anger_number + 1
    elif filenum <= 350:
      emo_sad_number = emo_sad_number + 1
  #print(emo_happy_number,emo_surprise_number,emo_neutral_number,emo_fear_number,emo_disgust_number, emo_anger_number,emo_sad_number) 

In [ ]:
#*감정별로 파일 분류코드 (ver3)

#data 폴더내의 모든 영상 파일을 file_list 변수에 불러옴.
#filenum = 001 형식.
#filename = 001-001.m2ts 형식.
#emo_happy = 001-001.m2ts 형식.

#분류 비 정하기
data_ratio = 0.8 
data_number = 10315 #폴더 내 영상 수
train_ratio = 1178 #1178
start = time.time()

path = '/content/drive/MyDrive/data'
file_list = os.listdir(path)
file_list.sort()

train_emo=[]
test_emo=[]
train_img = None
test_img=None

how_many_each_emo_number()
emo_happy_starter = 1
emo_surprise_starter = 1
emo_neutral_starter = 1
emo_fear_starter = 1
emo_disgust_starter = 1
emo_anger_starter = 1
emo_sad_starter = 1

for filename in file_list:
  filenum = filename[4:7]
  filenum = filenum.lstrip("0")
  filenum = int(filenum) #형식 변환
  if filenum <= 50:
    train_img, test_img, train_emo, test_emo, emo_happy_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,0,emo_happy_number,emo_happy_starter,filename)
  elif filenum <= 100:
    train_img, test_img, train_emo, test_emo, emo_surprise_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,1,emo_surprise_number,emo_surprise_starter,filename)
  elif filenum <= 150:
    train_img, test_img, train_emo, test_emo, emo_neutral_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,2,emo_neutral_number,emo_neutral_starter,filename)
  elif filenum <= 200:
    train_img, test_img, train_emo, test_emo, emo_fear_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,3,emo_fear_number,emo_fear_starter,filename)
  elif filenum <= 250:
    train_img, test_img, train_emo, test_emo, emo_disgust_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,4,emo_disgust_number,emo_disgust_starter,filename)
  elif filenum <= 300:
    train_img, test_img, train_emo, test_emo, emo_anger_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,5,emo_anger_number,emo_anger_starter,filename)
  elif filenum <= 350:
    train_img, test_img, train_emo, test_emo, emo_sad_starter = dataset_labeling(train_img,test_img, train_emo, test_emo,6,emo_sad_number,emo_sad_starter,filename)

np.save('/content/drive/MyDrive/save_numpy/train_img', train_img) # train_img.npy
np.save('/content/drive/MyDrive/save_numpy/test_img', test_img) # test_img.npy
np.save('/content/drive/MyDrive/save_numpy/train_emo', train_emo) # train_emo.npy
np.save('/content/drive/MyDrive/save_numpy/test_emo', test_emo) # test_emo.npy